In [1]:
import findspark
findspark.init()
import sys
import pyspark

In [2]:
sc = pyspark.SparkContext("yarn-client", "Parametrizacion")
from pyspark.sql import HiveContext
hiveContext = HiveContext(sc)

In [3]:
fileName = "/data_lake/Spark_Citi/input/parametros_citi_destinos.json" 
data = hiveContext.read.format("json").option('encoding', 'UTF-8').load(fileName)
data2 = data.rdd.collectAsMap()





In [4]:
data = hiveContext.read.format("json").option('encoding', 'UTF-8').load(fileName)
data.show()

+----------+--------------------+
|       key|                name|
+----------+--------------------+
|  BDorigen|          spark_data|
| BDdestino|        Citibanamex2|
|tblEcobici|             ecobici|
|    ModoOR|           overwrite|
|   ModoApp|              append|
|      tblH|     ecobici_hombres|
|      tblM|     ecobici_mujeres|
|   CamposM|genero_usuario,ed...|
|   CamposH|genero_usuario,ed...|
+----------+--------------------+



In [5]:
sc.broadcast(data2).value

{u'BDdestino': u'Citibanamex2',
 u'BDorigen': u'spark_data',
 u'CamposH': u'genero_usuario,edad_usuario,ciclo_estacion_retiro',
 u'CamposM': u'genero_usuario,edad_usuario,fecha_retiro,hora_retiro',
 u'ModoApp': u'append',
 u'ModoOR': u'overwrite',
 u'tblEcobici': u'ecobici',
 u'tblH': u'ecobici_hombres',
 u'tblM': u'ecobici_mujeres'}

In [91]:
def carga_vars():
    global bd_destino,bd_origen,modoOR,modoApp,tblH,tblM,camposH,camposM,tblEcobici   
    bd_destino = sc.broadcast(data2).value.get('BDdestino').encode('latin1')
    bd_origen = sc.broadcast(data2).value.get('BDorigen').encode('latin1')
    tblEcobici = sc.broadcast(data2).value.get('tblEcobici').encode('latin1')
    modoOR = sc.broadcast(data2).value.get('ModoApp').encode('latin1')
    modoApp = sc.broadcast(data2).value.get('ModoOR').encode('latin1')
    tblH = sc.broadcast(data2).value.get('tblH').encode('latin1')
    tblM = sc.broadcast(data2).value.get('tblM').encode('latin1')
    camposM = sc.broadcast(data2).value.get('CamposM').encode('latin1')
    camposH = sc.broadcast(data2).value.get('CamposH').encode('latin1') 
    
    return (bd_destino,bd_origen,modoOR,modoApp,tblH,tblM,camposH,camposM,tblEcobici)



def carga_listas():
    global list_h,list_m
    list_h = []
    list_m = []
    for x_ in camposM.split(","):
        list_h.append('' + x_.lower() + '')
        
    for x_ in camposM.split(","):
        list_m.append('' + x_.lower() + '')

    return(list_h,list_m)

In [94]:
carga_vars()
carga_listas()

(['genero_usuario', 'edad_usuario', 'fecha_retiro', 'hora_retiro'],
 ['genero_usuario', 'edad_usuario', 'fecha_retiro', 'hora_retiro'])

In [118]:
tablaHombres = hiveContext.table(bd_origen +"." + tblEcobici).select(list_h)

In [119]:
tablaHombres.printSchema()

root
 |-- genero_usuario: string (nullable = true)
 |-- edad_usuario: string (nullable = true)
 |-- fecha_retiro: string (nullable = true)
 |-- hora_retiro: string (nullable = true)



In [115]:
lista=("genero_usuario","edad_usuario","fecha_retiro","hora_retiro")
tablaHombres2 = hiveContext.table(bd_origen +"." + tblEcobici).select(list_h)